# Uso de Pipeline

[`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) puede utilizarse para encadenar varios estimadores en uno solo. Esto es útil porque a menudo hay una secuencia fija de pasos en el procesamiento de los datos, por ejemplo, selección de características, normalización y clasificación. En este caso, [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) tiene varias finalidades:

**Comodidad y encapsulación**: sólo tiene que llamar a [`fit`](https://scikit-learn.org/stable/glossary.html#term-fit) y [`predict`](https://scikit-learn.org/stable/glossary.html#term-predict) una vez en los datos para ajustar toda una secuencia de estimadores.

**Selección conjunta de parámetros**: [`grid search`](https://scikit-learn.org/stable/modules/grid_search.html#grid-search) sobre los parámetros de todos los estimadores del pipeline a la vez.

**Seguridad**: los pipelines ayudan a evitar la filtración de estadísticas de los datos de prueba al modelo entrenado en el cross-validation, garantizando que se utilicen las mismas muestras para entrenar los transformadores y los predictores.

Todos los estimadores de un pipeline, excepto el último, deben ser transformadores (es decir, deben tener un método [`transform`](https://scikit-learn.org/stable/glossary.html#term-transform)). El último estimador puede ser de cualquier tipo (transformador, clasificador, etc.).

In [1]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.decomposition import PCA
estimators = [("reduce_dim", PCA()), ("clf", SVC())]
pipe = Pipeline(estimators)
pipe

Pipeline(steps=[('reduce_dim', PCA()), ('clf', SVC())])

Otra forma de generar un pipeline es mediante la función [`make_pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html#sklearn.pipeline.make_pipeline), en este los nombres de los pasos se llenan automáticamente.

In [2]:
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import Binarizer
make_pipeline(Binarizer(), MultinomialNB())

Pipeline(steps=[('binarizer', Binarizer()), ('multinomialnb', MultinomialNB())])

Los estimadores de un pipeline se guardan en una lista dentro del atributo `steps`, pero se pueden acceder por índice o por el nombre del paso desde el propio Pipeline.

In [3]:
pipe.steps[0]

('reduce_dim', PCA())

In [4]:
pipe[0]

PCA()

In [5]:
pipe["reduce_dim"]

PCA()

Los parámetros de los estimadores en el pipelina se pueden acceder usando el syntax `<estimator>__<parameter>`.

In [6]:
pipe.set_params(clf__C=10)

Pipeline(steps=[('reduce_dim', PCA()), ('clf', SVC(C=10))])

Esto se utiliza principalmente para usar Grid Search.

In [7]:
from sklearn.model_selection import GridSearchCV
param_grid = dict(
    reduce_dim__n_components=[2, 5, 10],
    clf__C=[0.1, 10, 100]
)
grid_search = GridSearchCV(pipe, param_grid=param_grid)

Los pasos individuales también pueden sustituirse por parámetros, y los pasos no finales pueden ser ignorados usando el término `"passthrough"`.

In [8]:
from sklearn.linear_model import LogisticRegression
param_grid = dict(
    reduce_dim=["passthrough", PCA(5), PCA(10)],
    clf=[SVC(), LogisticRegression()],
    clf__C=[0.1, 10, 100]
)
grid_search = GridSearchCV(pipe, param_grid=param_grid)

Generamos un pipeline para el entrenamiento de un modelo donde se realiza una selección de características y el entrenamiento del modelo.

In [9]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
iris = load_iris()
pipe = Pipeline(steps=[
   ("select", SelectKBest(k=2)),
   ("clf", LogisticRegression())
])
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)
pipe.fit(X_train, y_train)
print(accuracy_score(y_test, pipe.predict(X_test)))

1.0


## Pre-procesamiento y clasificación

Se puede anidar pasos de pre-procesamiento y clasificación en un Pipeline.

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import set_config

steps = [
    ("preprocessing", StandardScaler()),
    ("classifier", LogisticRegression()),
]
pipe = Pipeline(steps)
set_config(display="diagram")
pipe

Pipeline(steps=[('preprocessing', StandardScaler()),
                ('classifier', LogisticRegression())])

## Múltiples pasos de pre-procesamienyo y clasificación

Se puede hacer más de un paso de pre-procesamiento en los Pipelines.

In [22]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression

steps = [
    ("standard_scaler", StandardScaler()),
    ("polynomial", PolynomialFeatures(degree=3)),
    ("classifier", LogisticRegression(C=2.0)),
]
pipe = Pipeline(steps)
pipe

Pipeline(steps=[('standard_scaler', StandardScaler()),
                ('polynomial', PolynomialFeatures(degree=3)),
                ('classifier', LogisticRegression(C=2.0))])

## Reducción de dimensionalidad y clasificación

Además de pasos de pre-procesamiento también se puede realizar un proceso de reducción de dimensionalidad.

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.decomposition import PCA

steps = [
    ("reduce_dim", PCA(n_components=4)),
    ("classifier", SVC(kernel="linear"))
]
pipe = Pipeline(steps)
pipe

Pipeline(steps=[('reduce_dim', PCA(n_components=4)),
                ('classifier', SVC(kernel='linear'))])

## Pipeline complejo con Column Transformer

Sabemos que en un dataset no todas las características son numéricas, también pueden ser categóricas, para este tipo de casos se puede generar un Pipeline más complejo donde se aniden dos Pipelines con pre-procesamientos distintos para cada tipo de dato.

In [24]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression

numeric_preprocessor = Pipeline(
    steps=[
        ("imputation_mean", SimpleImputer(missing_values=np.nan, strategy="mean")),
        ("scaler", StandardScaler()),
    ]
)

categorical_preprocessor = Pipeline(
    steps=[
        ("imputation_constant", SimpleImputer(fill_value="missing", strategy="constant")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)

preprocessor = ColumnTransformer(
    [
        ("categorical", categorical_preprocessor, ["state", "gender"]),
        ("numerical", numeric_preprocessor, ["age", "weight"]),
    ]
)

pipe = make_pipeline(preprocessor, LogisticRegression(max_iter=500))
pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('imputation_constant',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['state', 'gender']),
                                                 ('numerical',
                                                  Pipeline(steps=[('imputation_mean',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'weight'])])),
                ('logisticregression', LogisticRegression(max_iter=500))])

## Optimización de hiperparámetros sobre un pipeline y clasificador

También se puede realizar un proceso de optimización de hiperparámetros sobre los pasos de un pipeline y en el clasificador.

In [26]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

numeric_preprocessor = Pipeline(
    steps=[
        ("imputation_mean", SimpleImputer(missing_values=np.nan, strategy="mean")),
        ("scaler", StandardScaler()),
    ]
)

categorical_preprocessor = Pipeline(
    steps=[
        ("imputation_constant", SimpleImputer(fill_value="missing", strategy="constant")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)

preprocessor = ColumnTransformer(
    [
        ("categorical", categorical_preprocessor, ["state", "gender"]),
        ("numerical", numeric_preprocessor, ["age", "weight"]),
    ]
)

pipe = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", RandomForestClassifier())]
)

param_grid = {
    "classifier__n_estimators": [200, 500],
    "classifier__max_features": ["auto", "sqrt", "log2"],
    "classifier__max_depth": [4, 5, 6, 7, 8],
    "classifier__criterion": ["gini", "entropy"],
}

grid_search = GridSearchCV(pipe, param_grid=param_grid, n_jobs=-1)
grid_search

GridSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('categorical',
                                                                         Pipeline(steps=[('imputation_constant',
                                                                                          SimpleImputer(fill_value='missing',
                                                                                                        strategy='constant')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['state',
                                                                          'gender']),
                                                                        ('numerical',
                                                                         Pipeline(steps=[('imputation_mean',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['age',
                                                                          'weight'])])),
                                       ('classifier',
                                        RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'classifier__criterion': ['gini', 'entropy'],
                         'classifier__max_depth': [4, 5, 6, 7, 8],
                         'classifier__max_features': ['auto', 'sqrt', 'log2'],
                         'classifier__n_estimators': [200, 500]})

# Selección de características con Pipeline

Generamos un dataset de clasificación binaria con 20 características, donde 3 características son informativas y sin características redundantes.

In [10]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(
    n_features=20,
    n_informative=3,
    n_redundant=0,
    n_classes=2,
    n_clusters_per_class=2,
    random_state=42,
)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

Un error común es realizar el feature selection para el dataset completo en lugar del set de entrenamiento. Usar Pipeline previene este tipo de errores.

En este caso el primer paso consiste en el feature selection. Cuando se llama al método `fit` en la data de entrenamiento, un subconjunto de las características serán seleccionadas y el índice de las características seleccionadas serán almacenados. Este subconjunto de características son ingresadas al clasificador que luego será entrenado.

In [11]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC

anova_filter = SelectKBest(f_classif, k=3)
clf = LinearSVC()
anova_svm = make_pipeline(anova_filter, clf)
anova_svm.fit(X_train, y_train)

Pipeline(steps=[('selectkbest', SelectKBest(k=3)), ('linearsvc', LinearSVC())])

Ahora que el entrenamiento se completó, podemos predecir en muestras nuevas. En este caso el selector de características solo selecciona las características más discriminativas con base en la información guardad durante el entrenamiento. Luego la data entra al clasificador que luego hace la predicción.

In [12]:
from sklearn.metrics import classification_report

y_pred = anova_svm.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.80      0.86        15
           1       0.75      0.90      0.82        10

    accuracy                           0.84        25
   macro avg       0.84      0.85      0.84        25
weighted avg       0.85      0.84      0.84        25



# PCA, regresión logística y optimización de hiperparámetros con Pipeline

In [16]:
# Librerías
import numpy as np
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

# Pasos del proyecto
pca = PCA()
scaler = StandardScaler()
logistic = LogisticRegression(
    max_iter=10000,
    tol=0.1
)

# Generamos el pipeline
pipe = Pipeline(
    steps=[
        ("scaler", scaler),
        ("pca", pca),
        ("logistic", logistic)
    ]
)

# Cargamos la data
X_digits, y_digits = datasets.load_digits(return_X_y=True)

# Definimos la grilla de parámetros
param_grid = {
    "pca__n_components": [5, 15, 30, 45, 60],
    "logistic__C": np.logspace(-4, 4, 4),
}

# Optimización de hiperparámetros
search = GridSearchCV(pipe, param_grid, n_jobs=-1)
search.fit(X_digits, y_digits)

print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

Best parameter (CV score=0.924):
{'logistic__C': 0.046415888336127774, 'pca__n_components': 60}
